In [238]:
Infamily = ["Molly Weasley", "Arthur Weasley", "Ron Weasley"]
Outfamily = ["Harry Potter", "Albus Dumbledore", "Severus Snape", "Hermione Granger"]
total = Infamily + Outfamily
print(len(total))
total1 = []
for x in total:
    total1.append(x.split()[0])
    total1.append(x.split()[1])
total1 = list(set(total1))
total

7


['Molly Weasley',
 'Arthur Weasley',
 'Ron Weasley',
 'Harry Potter',
 'Albus Dumbledore',
 'Severus Snape',
 'Hermione Granger']

In [239]:
# all pairs generated from the 8 total names
# and corresponding labels
pairs = []
label1 = []
for x in total:
    for y in total:
        pairs.append(x+","+y)
        if x in Infamily and y in Infamily:
            label1.append("1")
        else:
            label1.append("0")
pairs = "\n".join(pairs)
label1 = "\n".join(label1)      
with open("sample_namepair.txt","w") as f:
    f.write(pairs)
with open("sample_label.txt","w") as f:
    f.write(label1)

In [240]:
# split sentences
import codecs
import nltk.data
# nltk.download('punkt')
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

with codecs.open("book_1.txt",'r',encoding='utf8') as f:
    data = f.read()
    script = '\n'.join(tokenizer.tokenize(data))
    script = script.encode("utf8")
with open("book_1_1.txt","w") as f:
    f.write(script)

In [323]:
# extract sentences contains the names
# removed names from the sentences
import fileinput
import re
script2 = []
label = []
name_pair = []
for line in fileinput.input("book_1_1.txt"):
    line1 = re.findall(r"[\w']+", line)
    for x in total:
        for y in total[(total.index(x)+1):]:
            #if x.split()[0] in line and y.split()[0] in line:
            if (x.split()[0].lower() + ' ' in line.lower() or x.split()[1].lower()
                + ' ' in line.lower()) and (y.split()[0].lower() + ' ' in line.lower()
                or y.split()[1].lower() + ' ' in line.lower()):
                line2 = [w for w in line1 if not w in total1]
                line2 = " ".join(line2)
                #print(str(line2))
                name_pair.append((x,y))
                script2.append(str(line2))
                if x in Infamily and y in Infamily:
#                     print(x, y)
#                     print("pos")
                    label.append("pos")
                else:
#                     print(x,y)
#                     print("neg")
                    label.append("neg")
fileinput.close()

#script2 = sorted(set(script2), key=script2.index)
script3 = "\n".join(script2)
with open("book_1_2.txt","w") as f:
    f.write(script3)

In [326]:
script4 = []
with codecs.open("book_1_2.txt",'r',encoding='utf8') as f:
    for line in f:
        script4.append(line)
len(script4)

300

In [327]:
import random
# sentense, labels and name pair
sample3 = list(zip(script4, label, name_pair))
random.seed(1)
sample = random.sample(sample3, 50)
# sentense and labels
sample2 = []
for x in sample3:
    y = (x[0], x[1])
    sample2.append(y)
# train: sentense and labels| train_name_pair
train = []
train_name_pair = []
for x in sample:
    y = (x[0], x[1])
    train.append(y)
    train_name_pair.append(x[2])

# test3: sentense, lables and test_name_ pair
test3 = [x for x in sample3 if x not in sample]
# test2: sentense and labels for test
# isolated test name pair
test2 = []
test_name_pair = []
for x in test3:
    test_name_pair.append(x[2])
    y = (x[0], x[1])
    test2.append(y)
# isolated test label
test_label = []
for x in test3:
    test_label.append(x[1])
test_label1 = "\n".join(test_label)
with open("test_label.txt", "w") as f:
    f.write(test_label1)

In [185]:
from textblob.classifiers import DecisionTreeClassifier

In [328]:
# train classifier
import time
start_time = time.time()
cl_tree = DecisionTreeClassifier(train)
print("--- %s seconds ---" % (time.time() - start_time))

--- 1.54732179642 seconds ---


In [329]:
# classification
import time
start_time = time.time()
label_test_tree = []
feature_tree = []
for x in test2:
    l = cl_tree.classify(x[0])
    f = cl_tree.extract_features(x[0])
    label_test_tree.append(l)
    feature_tree.append(f)
print("--- %s seconds ---" % (time.time() - start_time))

--- 14.8623299599 seconds ---


In [310]:
feature_tree[3]

{u'contains(And)': False,
 u'contains(As)': False,
 u'contains(Beaters)': False,
 u'contains(Bludger)': False,
 u'contains(Bludgers)': False,
 u'contains(Bogies)': False,
 u'contains(Both)': False,
 u'contains(Charlie)': False,
 u'contains(Cloak)': False,
 u'contains(Crabbe)': False,
 u'contains(Curse)': False,
 u'contains(Dumbledore)': False,
 u'contains(Dursleys\u2019)': False,
 u'contains(Famous)': False,
 u'contains(Filch)': False,
 u'contains(Flamel)': False,
 u'contains(Fred)': False,
 u'contains(George)': False,
 u'contains(Goyle)': False,
 u'contains(Granger)': False,
 u'contains(Great)': False,
 u'contains(Gryf\xadfindor)': False,
 u'contains(Hagrid)': False,
 u'contains(Hall)': False,
 u'contains(Halloween)': False,
 u'contains(Harry)': True,
 u'contains(Harry\u2019s)': False,
 u'contains(He)': False,
 u'contains(Hermione)': False,
 u'contains(Hufflepuff)': False,
 u'contains(Invisibility)': False,
 u'contains(Jordan)': False,
 u'contains(Lavender)': False,
 u'contains(Lee)':

In [330]:
label_test1 = "\n".join(label_test_tree)
with open("test_pred_tree.txt", "w") as f:
    f.write(label_test1)

In [331]:
pred = zip(test_name_pair, label_test_tree)
# in pred_label value, first element is accumulated prediction, the second element is total count of sentences for the 
# pairs, the third one is real relation
label_tree = {}
for x in pred:
    if x[0] in label_tree.keys():
        label_tree[x[0]][1] += 1
    else:
        label_tree[x[0]] = [0,1,0]
    if x[1] == "pos":
        label_tree[x[0]][0] += 1
    else:
        label_tree[x[0]][0] -= 1
    if x[0][0] in Infamily and x[0][1] in Infamily:
        label_tree[x[0]][2] = 1
    else:
        label_tree[x[0]][2] = 0

In [332]:
label_tree

{('Albus Dumbledore', 'Severus Snape'): [-1, 1, 0],
 ('Arthur Weasley', 'Harry Potter'): [1, 3, 0],
 ('Arthur Weasley', 'Hermione Granger'): [0, 2, 0],
 ('Arthur Weasley', 'Ron Weasley'): [13, 25, 1],
 ('Arthur Weasley', 'Severus Snape'): [-1, 1, 0],
 ('Harry Potter', 'Albus Dumbledore'): [-6, 6, 0],
 ('Harry Potter', 'Hermione Granger'): [-41, 41, 0],
 ('Harry Potter', 'Severus Snape'): [-17, 17, 0],
 ('Molly Weasley', 'Arthur Weasley'): [10, 22, 1],
 ('Molly Weasley', 'Harry Potter'): [1, 3, 0],
 ('Molly Weasley', 'Hermione Granger'): [0, 2, 0],
 ('Molly Weasley', 'Ron Weasley'): [9, 21, 1],
 ('Ron Weasley', 'Albus Dumbledore'): [-1, 1, 0],
 ('Ron Weasley', 'Harry Potter'): [-59, 65, 0],
 ('Ron Weasley', 'Hermione Granger'): [-31, 33, 0],
 ('Ron Weasley', 'Severus Snape'): [-4, 4, 0],
 ('Severus Snape', 'Hermione Granger'): [-3, 3, 0]}

In [333]:
# accuracy
j = 0
print(len(label_test_tree))
print(len(test_label))
for i in range(len(test2)):
    if test2[i][1] != label_test_tree[i]:
        print(test3[i][2])
        j += 1
print(j/float(i))
print(j)

250
250
('Molly Weasley', 'Harry Potter')
('Arthur Weasley', 'Harry Potter')
('Ron Weasley', 'Harry Potter')
('Molly Weasley', 'Hermione Granger')
('Arthur Weasley', 'Hermione Granger')
('Ron Weasley', 'Hermione Granger')
('Ron Weasley', 'Harry Potter')
('Molly Weasley', 'Harry Potter')
('Arthur Weasley', 'Harry Potter')
('Ron Weasley', 'Harry Potter')
('Molly Weasley', 'Arthur Weasley')
('Molly Weasley', 'Ron Weasley')
('Arthur Weasley', 'Ron Weasley')
('Molly Weasley', 'Arthur Weasley')
('Molly Weasley', 'Ron Weasley')
('Arthur Weasley', 'Ron Weasley')
('Molly Weasley', 'Arthur Weasley')
('Molly Weasley', 'Ron Weasley')
('Arthur Weasley', 'Ron Weasley')
('Molly Weasley', 'Arthur Weasley')
('Molly Weasley', 'Ron Weasley')
('Arthur Weasley', 'Ron Weasley')
('Molly Weasley', 'Arthur Weasley')
('Molly Weasley', 'Ron Weasley')
('Arthur Weasley', 'Ron Weasley')
('Molly Weasley', 'Arthur Weasley')
('Molly Weasley', 'Ron Weasley')
('Arthur Weasley', 'Ron Weasley')
0.112449799197
28


In [216]:
i = 0
result = []
for x in test2:
#     print(type(x[0]))
#     print(type(x[1]))
#     print(type(label_test[i]))
#     print(type(x[0] + "," + x[1] + "," + label_test[i]))
    result.append(x[0] + "," + x[1] + "," + label_test[i])
    i += 1
result = "\n".join(result)
#print(type(result))

In [217]:
with open("classification",'w') as f:
    f.write(result.encode('utf8'))

In [289]:
# new sample for test
testname = []
testscript = []
# generate name pairs and sentences including them
for line in fileinput.input("book_1_1.txt"):
    line1 = re.findall(r"[\w']+", line)
    for x in testname:
        for y in testname[(testname.index(x)+1):]:
            if (x.split()[0].lower() + ' ' in line.lower() or x.split()[1].lower()
                + ' ' in line.lower()) and (y.split()[0].lower() + ' ' in line.lower()
                or y.split()[1].lower() + ' ' in line.lower()):
                line2 = [w for w in line1 if not w in total1]
                line2 = " ".join(line2)
                #print(str(line2))
                testname.append((x,y))
                testscript.append(str(line))
            for name1 in nick[x][1:]:
                for name2 in nick[y][1:]:
                    if name1.lower() + ' ' in line.lower() and name2.lower() + ' ' in line.lower():
                        family_matrix.loc[x,y] += 1
                        family_matrix.loc[y,x] += 1

fileinput.close()

In [ ]:
for x in total:
    for y in total[(total.index(x)+1):]:
        nick[x]

In [311]:
import pickle
with open("names_with_nick_ep1", "rb") as f:
    nick = pickle.load(f)

In [314]:
nick

{'"Nearly Headless" Nick': ['"Nearly', 'Headless"', 'Nick'],
 'Albus Dumbledore': ['Albus', 'Dumbledore', 'Percival', 'Wulfric', 'Brian'],
 'Alicia Spinnet': ['Alicia', 'Spinnet'],
 'Angelina Johnson': ['Angelina', 'Johnson'],
 'Argus Filch': ['Argus', 'Filch'],
 'Arthur Weasley': ['Arthur', 'Weasley'],
 'Bathilda Bagshot': ['Bathilda', 'Bagshot'],
 'Charlie Weasley': ['Charlie', 'Weasley'],
 'Cornelius Fudge': ['Cornelius', 'Fudge', 'Oswald'],
 'Dean Thomas': ['Dean', 'Thomas'],
 'Dedalus Diggle': ['Dedalus', 'Diggle'],
 'Draco Malfoy': ['Draco', 'Malfoy'],
 'Dudley Dursley': ['Dudley', 'Dursley'],
 'Fred Weasley': ['Fred', 'Weasley'],
 'George Weasley': ['George', 'Weasley'],
 'Ginevra Weasley': ['Ginevra', 'Weasley', 'Ginny', 'Molly'],
 'Gregory Goyle Jr': ['Gregory', 'Goyle', 'Jr'],
 'Gregory Goyle Sr': ['Gregory', 'Goyle', 'Sr'],
 'Hannah Abbott': ['Hannah', 'Abbott'],
 'Harry Potter': ['Harry', 'Potter', 'James'],
 'Helena Ravenclaw': ['Helena', 'Ravenclaw', 'The', 'Grey'],
 'Her